In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C://Users/igor-/OneDrive/Área de Trabalho/Kaggle_tests/MNIST/Kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C://Users/igor-/OneDrive/Área de Trabalho/Kaggle_tests/MNIST/Kaggle\sample_submission.csv
C://Users/igor-/OneDrive/Área de Trabalho/Kaggle_tests/MNIST/Kaggle\test.csv
C://Users/igor-/OneDrive/Área de Trabalho/Kaggle_tests/MNIST/Kaggle\train.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.datasets import mnist
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import RandomTranslation, RandomZoom, RandomRotation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, TerminateOnNaN
from tensorflow.keras.optimizers import Adam
from tensorflow.data import AUTOTUNE
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:


(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = np.concatenate((X_train, X_test))
y_train = np.concatenate((y_train, y_test))

valid = pd.read_csv('Kaggle/train.csv').to_numpy()
test = pd.read_csv('Kaggle/test.csv').to_numpy()

In [4]:
train_Y = tf.keras.utils.to_categorical(y_train)
valid_Y = tf.keras.utils.to_categorical(valid[:,0])

train_X = X_train / 255.0
valid_X = valid[:,1:] / 255.0
test_X = test / 255.0

train_X = train_X.reshape(-1, 28, 28, 1)
valid_X = valid_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

print(train_X.shape,valid_X.shape,test_X.shape)

(70000, 28, 28, 1) (42000, 28, 28, 1) (28000, 28, 28, 1)


In [5]:
BS = 128 
EPOCHS = 25
STEPS = len(train_X) // BS
LABELS = 10

In [6]:
train_DS = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
train_DS = train_DS.shuffle(STEPS * BS).batch(BS).repeat().prefetch(AUTOTUNE)

valid_DS = tf.data.Dataset.from_tensor_slices((valid_X, valid_Y))
valid_DS = valid_DS.batch(BS)